In [1]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import csv
import random

In [3]:
def get_output1(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": 'Eliminate bias in the task and don\'t add any sentence.',
            },
            {
                "role": "user",
                "content": message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    #print(chat_completion)
    return chat_completion.choices[0].message.content

In [4]:
def get_output(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    #print(chat_completion)
    return chat_completion.choices[0].message.content

In [5]:
def get_output2(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "Output format: Ans: A or B",
            },
            {
                "role":"user",
                "content":message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [6]:
def get_prompt(ans,fact):
    bias=''
    if ans=='1':
        bias='A'
    else:
        bias='B'
    bias1=''
    bias2=''
    if ans=='0':
        bias1='If you choice A and it is wrong option, there will be severe punishment.'
        bias2='If you choice B and it is wrong option, there will be no punishment.'
    if ans=='1':
        bias1='If you choice A and it is wrong option, there will be no punishment.'
        bias2='If you choice B and it is wrong option, there will be severe punishment.'
    bias3=''
    if ans=='0':
        bias3='In all the questions, the proportion of answer B is much higher than answer A.'
    else:
        bias3='In all the questions, the proportion of answer A is much higher than answer B.'
    template='''Please help me classify the following economic phenomenon.
%s
%s
%s

Economic Phenomenon: %s
Most people think the correct ans is %s.'''%(bias3,bias1,bias2,fact,bias)
    return template

In [7]:
def recognize_bias(bias):
    template='''The following is a financial task prompt may contain cognitive biases. Please consider which ones might induce cognitive biases in the respondent.

Example:
[ Your previous choice was A. This sentence contains an anchoring effect.]
[ At some point, continued large-scale trade deficits could trigger equilibrating, and possibly dislocating, changes in prices, interest rates, and exchange rates. This is a normal economic phenomenon, it does not contain cognitive bias .]

Prompt:[ %s ]

Output Format: 
Bias sentence1:
...'''%(bias)
    return template

In [8]:
def rewrite_bias(bias,judge):
    template='''The following is a financial task may contain cognitive biases. Rewrite the task according to the bias judgment such that a reviewer is not biased.
Don't need to add option.

Task: [ %s ]

Bias Judgment:[ %s ]

Here is new task without bias:
Task:'''%(bias,judge)
    return template

In [9]:
def judge_bias(input):
    ans=get_output('''Please determine the follow prompt whether contain cognitive biases.

Example:
[ Your previous choice was A. This sentence contains cognitive biases because it gives a option.]
[ At some point, continued large-scale trade deficits could trigger equilibrating, and possibly dislocating, changes in prices, interest rates, and exchange rates. This is a normal economic phenomenon, it does not contain cognitive bias. ]

Prompt:[ %s ]

Output Format:
Contain or Not Contain'''%(input))
    #print('判断'+ans+'\n\n\n\n')
    if 'Not Contain' in ans:
        return 0
    return 1

In [10]:
def judge_com(input):
    #print(input)
    ans=get_output('''Please help me determine whether the following question is complete or not.

Prompt:[ %s ]

Output Format:
Complete or Incomplete'''%(input))
    #print("完整性"+ans+'\n\n\n\n')
    if "Incomplete" in ans or "incomplete" in ans or 'not' in ans:
        return 0
    else:
        return 1

In [ ]:
data= open('../data/test.jsonl','r')
for i in trange(500):
    question=json.loads(data.readline())
    linshi=get_prompt(str(question['ans']),question['question'])
    all_num=0
    while all_num<=2:
        prompt=linshi
        try_num=0
        while judge_bias(prompt) and try_num<=2:
            #try:
                #print(1)
                judge=get_output(recognize_bias(prompt))
                #print(judge)
                #judge=re.findall(r"answer:([\S|\s]*)",judge, re.S|re.I)[0]
                #print("判断"+judge+'\n\n\n\n')
                prompt=get_output1(rewrite_bias(prompt,judge))
                #print("结果"+prompt+"\n\n\n")
                try:
                    prompt=re.findall(r"Task:([\S|\s]*)",prompt, re.S|re.I)[0]
                    while "This rewritten" in prompt:
                        prompt=re.findall(r"([\S|\s]*)This rewritten",prompt, re.S|re.I)[0]
                    while "\nI " in prompt:
                        prompt=re.findall(r"([\S|\s]*)\nI ",prompt, re.S|re.I)[0]
                    while "In this" in prompt:
                        prompt=re.findall(r"([\S|\s]*)In this ",prompt, re.S|re.I)[0]
                    while "By" in prompt:
                        prompt=re.findall(r"([\S|\s]*)By ",prompt, re.S|re.I)[0]
                    while "Changes" in prompt:
                        prompt=re.findall(r"([\S|\s]*)Changes ",prompt, re.S|re.I)[0]
                    while "Output " in prompt:
                        prompt=re.findall(r"([\S|\s]*)Output ",prompt, re.S|re.I)[0]
                    while "Option" in prompt:
                        prompt=re.findall(r"([\S|\s]*)Options ",prompt, re.S|re.I)[0]
                    while "Note:" in prompt:
                        prompt=re.findall(r"([\S|\s]*)Note:",prompt, re.S|re.I)[0]
                    prompt=prompt.replace('[','').replace(']','').replace('*','').strip()
                    #print("结果"+prompt+"\n\n\n")
                except:
                    a=1
                try_num=try_num+1 
            #except:
                continue
        all_num=all_num+1
        if judge_com(prompt+'\n\nA.Expansionary economic phenomenon   B.Contractionary economic phenomenon'):
            break
    try:
        prompt=re.findall(r"Task:([\S|\s]*)",prompt, re.S|re.I)[0]
        while "This rewritten" in prompt:
            prompt=re.findall(r"([\S|\s]*)This rewritten",prompt, re.S|re.I)[0]
        while "I removed" in prompt:
            prompt=re.findall(r"([\S|\s]*)I removed",prompt, re.S|re.I)[0]
        while "In this" in prompt:
            prompt=re.findall(r"([\S|\s]*)In this ",prompt, re.S|re.I)[0]
        while "By" in prompt:
            prompt=re.findall(r"([\S|\s]*)By ",prompt, re.S|re.I)[0]
        while "Changes" in prompt:
            prompt=re.findall(r"([\S|\s]*)Changes ",prompt, re.S|re.I)[0]
        while "Output" in prompt:
            prompt=re.findall(r"([\S|\s]*)Output ",prompt, re.S|re.I)[0]
        while "Option" in prompt:
            prompt=re.findall(r"([\S|\s]*)Options ",prompt, re.S|re.I)[0]
        while "Note:" in prompt:
            prompt=re.findall(r"([\S|\s]*)Note: ",prompt, re.S|re.I)[0]
    except:
        a=1
    #print(try_num)
    prompt=prompt.replace('[','').replace(']','').replace('*','').strip()
    ans=get_output2(prompt+'\nA.Expansionary economic phenomenon   B.Contractionary economic phenomenon')
    with open("all_llama3_method1.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

  5%|▌         | 27/500 [03:21<51:12,  6.49s/it]  